In [9]:
from langchain_community.llms import Ollama
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import TextLoader
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.runnables import RunnablePassthrough, RunnableParallel


In [10]:
# Загрузка модели Llama
ollama_llm = Ollama(model='llama3')

In [11]:
# Инициализация объектов
parser = StrOutputParser()
# Создание цепочки обработки данных для Llama3
llama_chain = ollama_llm | parser

In [12]:
# Загрузка и обработка документа
loader = TextLoader('data1.txt', encoding='utf-8')
document = loader.load()
# Разделение документа на часи
spliter = RecursiveCharacterTextSplitter(chunk_size=200, chunk_overlap=50)
chunks = spliter.split_documents(document)

In [13]:
# Вызов модели Llama3 для ответа на вопрос
# llama_chain.invoke('what is a bot')

In [14]:
# Создание векторного хранилища и поисковика на основе HuggingFaceEmbeddings
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vector_storage = FAISS.from_documents(chunks, embedding_model)
retriever = vector_storage.as_retriever()

/home/user/Desktop/OBJECT_DETECT_AND_LLM/Lama3-LangChain-RAG/venv/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [15]:
# Пример запроса к поисковику
# retriever.invoke('Инструкцию к какому прибору ты хранишь?')

In [16]:
# Создание шаблона запроса
template = """
You are AI-powered chatbot designed to provide 
information and assistance for customers
based on the context provided to you only. 

Context:{context}
Question:{question}
"""

In [17]:
prompt = PromptTemplate.from_template(template=template)
formatted_prompt = prompt.format(
    context=' Here is a context to use',
    question=' This is a question to answer'
)

In [18]:
# Создание цепочки запросов
result = RunnableParallel(context=retriever, question=RunnablePassthrough())
chain = result | prompt | ollama_llm | parser

In [19]:
# Примеры запросов к цепочке
# print(chain.invoke('общие сведения инструкции по эксплуатации'))
print(chain.invoke('что содержит твой контекст? отвечай на русском языке'))

Мои сведения основаны на четырех документах, которые содержат информацию о безопасном использовании ручного миксера. В этих документах есть указания о необходимости соблюдения мер предосторожности при использовании прибора, а также информации о техническом уходе и хранении.

В частности, в контексте содержится информация об опасностях, связанных с несоблюдением мер предосторожности при работе с миксером. К примеру, есть указание на то, что несоблюдение указаний может привести к травмам или опасности для жизни.

Также в контексте содержится информация о хранении и уходе за миксером, а также об использовании его аксессуаров. В целом, моя задача - помочь пользователю безопасно использовать миксер и выполнять работы, связанные с ним.
